# Pairwise conservative kernel for multiple species

In [1]:
# import libraries
import theano
import theano.tensor as T
from theano import function
import numpy as np
import datetime

## Step by step kernel construction

### Definitions

Define the position of the atoms relative to the centrla one, and their chemical species, by a matrix of dimension Mx5

In [2]:
# positions of central atoms
r1, r2 = T.dvectors('r1d','r2d') 
# positions of neighbours 
rho1, rho2 = T.dmatrices('rho1','rho2') 
# hyperparameter
sig = T.dscalar('sig') 

# positions of neighbours without chemical species

rho1s = rho1[:, 0:3]
rho2s = rho2[:, 0:3]

alpha_1 = rho1[:, 3:4].flatten()
alpha_2 = rho2[:, 3:4].flatten()
alpha_j = rho1[:, 4:5].flatten()
alpha_m = rho2[:, 4:5].flatten()


Calculate the relative distance to the central vector

In [3]:
# first configuration
r1j = T.sqrt(T.sum((rho1s[:, :]-r1[None,:])**2, axis = 1))
# second configuration
r2m = T.sqrt(T.sum((rho2s[:, :]-r2[None,:])**2, axis = 1))


Calculate the mask given by the chemical species

In [4]:
# numerical kronecker
def delta_alpha2(a1j, a2m):
    
    d = np.exp(-(a1j - a2m)**2/(2*(1e-5)**2))
    #d = np.exp((a1j[0] - a2m[0])**2/(2*(1e-7)**2))
    #dp = np.exp((a1j[1] - a2m[1])**2/(2*(1e-7)**2))
    
    return d

delta_alpha2(100, 100)

1.0

In [5]:
delta_alphas12 = delta_alpha2(alpha_1[:, None] , alpha_2[None, :])
delta_alphasjm = delta_alpha2(alpha_j[:, None] , alpha_m[None, :])
delta_alphas1m = delta_alpha2(alpha_1[:, None] , alpha_m[None, :])
delta_alphasj2 = delta_alpha2(alpha_j[:, None] , alpha_2[None, :])

### Cutoff function

In [6]:
theta = T.dscalar('theta')
rc = T.dscalar('rc')


In [11]:
theano.config.device

'cpu'

In [13]:
k_ij = (T.exp(-(r1j[:, None] - r2m[None,:])**2/sig) * (delta_alphas12 * delta_alphasjm + delta_alphas1m * delta_alphasj2))
k_ij = k_ij*(T.exp(-theta/(rc-r1j[:, None]))*T.exp(-theta/(rc-r2m[None,:])))/(T.exp(-theta/(rc-r1j[:, None]))*T.exp(-theta/(rc-r2m[None,:])))
# kernel
k = T.sum(k_ij)   

# energy energy kernel
k_ee_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k)

# energy force kernel
k_ef = T.grad(k, r2)
k_ef_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ef)

# force force kernel
k_ff = T.grad(k, r1)
k_ff_der, updates = theano.scan(lambda j, k_ff, r2 : T.grad(k_ff[j], r2), 
                                sequences = T.arange(k_ff.shape[0]), non_sequences = [k_ff, r2])

k_ff_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ff_der, on_unused_input='ignore')


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x,x}(TensorConstant{2})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/opt.py", line 2019, in process_node
    replacements = lopt.transform(node)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/tensor/opt.py", line 6507, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/cc.py", line 1217, in make_thunk
    keep_lock=keep_lock)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-p


You can find the C code in this temporary file: /var/folders/hr/4m44n3ss4fjfbzgdtznkl4q00000gn/T/theano_compilation_error_7z856od5


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x,x}(TensorConstant{-1.0})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/opt.py", line 2019, in process_node
    replacements = lopt.transform(node)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/tensor/opt.py", line 6507, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/cc.py", line 1217, in make_thunk
    keep_lock=keep_lock)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/sit


You can find the C code in this temporary file: /var/folders/hr/4m44n3ss4fjfbzgdtznkl4q00000gn/T/theano_compilation_error_g4oc2ovi


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x,x}(TensorConstant{4999999999.999999})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/opt.py", line 2019, in process_node
    replacements = lopt.transform(node)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/tensor/opt.py", line 6507, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/cc.py", line 1217, in make_thunk
    keep_lock=keep_lock)
  File "/Users/aldoglielmo/anaconda/lib/

...

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x,x}(TensorConstant{-4999999999.999999})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/opt.py", line 2019, in process_node
    replacements = lopt.transform(node)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/tensor/opt.py", line 6507, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/cc.py", line 1217, in make_thunk
    keep_lock=keep_lock)
  File "/Users/aldoglielmo/anaconda/lib


You can find the C code in this temporary file: /var/folders/hr/4m44n3ss4fjfbzgdtznkl4q00000gn/T/theano_compilation_error_tiur1k0_


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x,x}(TensorConstant{-1.0})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/opt.py", line 2019, in process_node
    replacements = lopt.transform(node)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/tensor/opt.py", line 6507, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/site-packages/theano/gof/cc.py", line 1217, in make_thunk
    keep_lock=keep_lock)
  File "/Users/aldoglielmo/anaconda/lib/python3.6/sit


You can find the C code in this temporary file: /var/folders/hr/4m44n3ss4fjfbzgdtznkl4q00000gn/T/theano_compilation_error_4gezhii3

You can find the C code in this temporary file: /var/folders/hr/4m44n3ss4fjfbzgdtznkl4q00000gn/T/theano_compilation_error_ffgk2gha


Exception: ('The following error happened while compiling the node', Subtensor{::, int64:int64:}(rho2, Constant{4}, Constant{5}), '\n', "Compilation failed (return status=1): In file included from /Users/aldoglielmo/.theano/compiledir_Darwin-17.4.0-x86_64-i386-64bit-i386-3.6.0-64/tmpizob9b3e/mod.cpp:1:. In file included from /Users/aldoglielmo/anaconda/include/python3.6m/Python.h:25:. /Users/aldoglielmo/anaconda/bin/../include/c++/v1/stdio.h:108:15: fatal error: 'stdio.h' file not found. #include_next <stdio.h>.               ^~~~~~~~~. 1 error generated.. ", '[Subtensor{::, int64:int64:}(rho2, Constant{4}, Constant{5})]')

### Saving the kernels

In [14]:
from six.moves import cPickle
import sys

sys.setrecursionlimit(3000)

f = open('2B_ee.save', 'wb')
cPickle.dump(k_ee_fun, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

f = open('2B_ef.save', 'wb')
cPickle.dump(k_ef_fun, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

f = open('2B_ff.save', 'wb')
cPickle.dump(k_ff_fun, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

NameError: name 'k_ee_fun' is not defined

## Construction of kernels over lists of configurations

In [15]:
import theano.typed_list as Tl

#### A simple example of a double for loop in Theano

In [22]:
# simple for loop 

a = T.scalar('a')
b = T.scalar('b')
prod = a*b
p_fun = function([a, b], prod)

list1, list2 = Tl.TypedListType(T.dscalar)('list1'), Tl.TypedListType(T.dscalar)('list2')
len1, len2 = Tl.length(list1), Tl.length(list2)
range1, range2 = T.arange(len1, dtype='int64'), T.arange(len2, dtype='int64')
mat = T.dmatrix('mat')

def fun2(j, e1, list2):
    
    return e1*list2[j]
    #return p_fun(e1,list2[j]) # THIS DOES NOT WORK!

def loop_over_list2(i, range2, list1, list2):
    
    e1 = list1[i]
    result_, updates_ = theano.scan(fn=fun2,
                              outputs_info = None,
                              non_sequences=[e1, list2],
                              sequences=[range2])
    return result_

result, updates = theano.scan(fn=loop_over_list2,
                              outputs_info = None,
                              non_sequences=[range2, list1, list2],
                              sequences=[range1])

f = theano.function(inputs=[list1, list2], outputs=result)

f([1, 2],[1, 3, 2])



You can find the C code in this temporary file: /var/folders/hr/4m44n3ss4fjfbzgdtznkl4q00000gn/T/theano_compilation_error__7xajtmr


Exception: ('The following error happened while compiling the node', Elemwise{mul,no_inplace}(a, b), '\n', "Compilation failed (return status=1): In file included from /Users/aldoglielmo/.theano/compiledir_Darwin-17.4.0-x86_64-i386-64bit-i386-3.6.0-64/tmp5ye_1d9i/mod.cpp:1:. In file included from /Users/aldoglielmo/anaconda/include/python3.6m/Python.h:25:. /Users/aldoglielmo/anaconda/bin/../include/c++/v1/stdio.h:108:15: fatal error: 'stdio.h' file not found. #include_next <stdio.h>.               ^~~~~~~~~. 1 error generated.. ", '[Elemwise{mul,no_inplace}(a, b)]')

### Define a symbolid "kernel" object and its derivative

In [ ]:
def kernel_ee_(r1, r2, rho1, rho2, sig, theta, rc):
    rho1s = rho1[:, 0:3]
    rho2s = rho2[:, 0:3]
    alpha_1 = rho1[:, 3:4].flatten()
    alpha_2 = rho2[:, 3:4].flatten()
    alpha_j = rho1[:, 4:5].flatten()
    alpha_m = rho2[:, 4:5].flatten()
    r1j = T.sqrt(T.sum((rho1s[:, :]-r1[None,:])**2, axis = 1))
    r2m = T.sqrt(T.sum((rho2s[:, :]-r2[None,:])**2, axis = 1))
    delta_alphas12 = delta_alpha2(alpha_1[:, None] , alpha_2[None, :])
    delta_alphasjm = delta_alpha2(alpha_j[:, None] , alpha_m[None, :])
    delta_alphas1m = delta_alpha2(alpha_1[:, None] , alpha_m[None, :])
    delta_alphasj2 = delta_alpha2(alpha_j[:, None] , alpha_2[None, :])
    k_ij = (T.exp(-(r1j[:, None] - r2m[None,:])**2/sig) * (delta_alphas12 * delta_alphasjm + delta_alphas1m * delta_alphasj2)
    *T.exp(-theta/(rc-r1j[:, None]))*T.exp(-theta/(rc-r2m[None,:])))
    k = T.sum(k_ij) 
    return k

def kernel_ef_(r1, r2, rho1, rho2, sig, theta, rc):
    k = kernel_ee_(r1, r2, rho1, rho2, sig, theta, rc)
    k_ef = T.grad(k, r2)

    return k_ef


def kernel_ff_(r1, r2, rho1, rho2, sig, theta, rc):
    k = kernel_ee_(r1, r2, rho1, rho2, sig, theta, rc)
    k_ff_der = T.grad(k, r1)
    k_ff, updates = theano.scan(lambda j, k_ff, r2 : T.grad(k_ff_der[j], r2), 
                                sequences = T.arange(k_ff_der.shape[0]), non_sequences = [k_ff_der, r2])

    return k_ff

### Define symbolic functions that loop over elements of a list

In [ ]:
def fun_ee(j, e1, list2, r1, r2, sig, theta, rc):
    return kernel_ee_(r1, r2, e1, list2[j], sig, theta, rc)

def fun_ff(j, e1, list2, r1, r2, sig, theta, rc):
    return kernel_ff_(r1, r2, e1, list2[j], sig, theta, rc)

def fun_ef(j, e1, list2, r1, r2, sig, theta, rc):
    return kernel_ef_(r1, r2, e1, list2[j], sig, theta, rc)


def loop_over_list_ee(i, range2, list1, list2, r1, r2, sig, theta, rc):
    e1 = list1[i]
    result_, updates_ = theano.scan(fn=fun_ee,
                              outputs_info = None,
                              non_sequences=[e1, list2, r1, r2,  sig, theta, rc],
                              sequences=[range2])
    return result_

def loop_over_list_ef(i, range2, list1, list2, r1, r2, sig, theta, rc):
    e1 = list1[i]
    result_, updates_ = theano.scan(fn=fun_ef,
                              outputs_info = None,
                              non_sequences=[e1, list2, r1, r2, sig, theta, rc],
                              sequences=[range2])
    return result_


def loop_over_list_ff(i, range2, list1, list2, r1, r2, sig, theta, rc):
    e1 = list1[i]
    result_, updates_ = theano.scan(fn=fun_ff,
                              outputs_info = None,
                              non_sequences=[e1, list2, r1, r2, sig, theta, rc],
                              sequences=[range2])
    return result_

### Define two list of configurations and the final functions

In [ ]:
# define lists of configurations to loop over (lists of matrices)

rho1l = Tl.TypedListType(T.dmatrix)('rho1l')
rho2l = Tl.TypedListType(T.dmatrix)('rho2l')

l1, l2 = Tl.length(rho1l), Tl.length(rho2l)
range1, range2 = T.arange(l1, dtype='int64'), T.arange(l2, dtype='int64')


In [ ]:
theano.config.openmp = False

In [ ]:
# lengths of the two lists

result, updates = theano.scan(fn=loop_over_list_ee,
                              outputs_info = None,
                              non_sequences=[range2, rho1l, rho2l, r1, r2, sig, theta, rc],
                              sequences=[range1])

k_ee_func_for = function(inputs=[rho1l, rho2l, r1, r2, sig, theta, rc], outputs=result)


In [ ]:

result, updates = theano.scan(fn=loop_over_list_ef,
                              outputs_info = None,
                              non_sequences=[range2, rho1l, rho2l, r1, r2, sig, theta, rc],
                              sequences=[range1])

k_ef_func_for = function(inputs=[rho1l, rho2l, r1, r2, sig, theta, rc], outputs=result)


In [ ]:

result, updates = theano.scan(fn=loop_over_list_ff,
                              outputs_info = None,
                              non_sequences=[range2, rho1l, rho2l, r1, r2, sig, theta, rc],
                              sequences=[range1])

k_ff_func_for = function(inputs=[rho1l, rho2l, r1, r2, sig, theta, rc], outputs=result)

## Tests

Test on a single configuration

In [ ]:
rho1n = np.array([[1, 2, 3, 1, 3],
                  [1, 2, 4, 1, 4],
                  [1, 2, 1, 1, 1]])

rho2n = np.array([[1.2, 2.1, 3.3, 1, 3],
                  [1.7, 2.1, 4.6, 1, 4],
                  [1.2, 2.1, 1.2, 1, 1]])

print(rho1n.shape , rho2n.shape)

In [ ]:
mat = k_ff_fun(np.array([0,0,0]), np.array([0,0,0]), rho1n, rho2n, 100., 1., 1.)
matT = k_ff_fun(np.array([0,0,0]), np.array([0,0,0]),rho2n, rho1n, 100., 1., 1.)


In [ ]:
print(mat)
print(np.isclose(mat,matT.T))

Tests on random configurations

In [ ]:
np.set_printoptions(precision=4)

In [ ]:
# create random configurations

n1 = 3
n2 = 3
d = 3 # this can't be changed
nn = 12
confs1 = (np.random.rand(n1, nn, d+2)-0.5)*100
confs2 = (np.random.rand(n2, nn, d+2)-0.5)*100

confs1[:, :, 3:5] = 1.
confs2[:, :, 3:5] = 1.

In [ ]:

ti_python = datetime.datetime.now()
gram = np.zeros((n1*d,n2*d))
for i in np.arange(n1):
    for j in np.arange(n2):
        gram[d*i:d*(i+1),d*j:d*(j+1)] = k_ff_fun(np.zeros(d),np.zeros(d), confs1[i],confs2[j], 10000., 1., 1000.)

tf_python = datetime.datetime.now()
(tf_python-ti_python).total_seconds()


In [ ]:

ti_theano = datetime.datetime.now()
gram_for = k_ff_func_for(confs1, confs2, np.zeros(3),np.zeros(3),  10000., 1., 1000.)
gram_for = np.reshape(gram_for.swapaxes(1, 2), (gram_for.shape[0]*gram_for.shape[2], gram_for.shape[1]*gram_for.shape[2]))
tf_theano = datetime.datetime.now()
(tf_theano-ti_theano).total_seconds()

In [ ]:
print(np.allclose(gram_for,gram))

In [ ]:
import matplotlib.pyplot as plt
eigs, eigvecs = np.linalg.eigh(gram)
print(np.min(eigs))
print(np.max(eigs))
plt.hist(eigs[:], 40)
plt.show()